In [ ]:
import datetime

start_time = datetime.datetime.now()

In [ ]:
import numpy as np
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [ ]:
def annual_median(zip_file):
    """Access a zip file and reads through the csv files in the zip file one by one.
        Computes median of the sum of the column 'total_obligation' according to year
        Returns the median, agency_id, file_size, and row_count of the file that was read in
    
    Args:
        zip_file (string)    : name of the zip file to be accessed
    
    Returns:
        median_list (list)   : List of the median values calculated
        agency_list (list)   : List of the agency_id's for each file
        file_size_list (list): List of the file_sizes for each file
        row_count_list (list): List of the row_counts for each file
    
    """
    award_zip = zipfile.ZipFile(zip_file)
    file_names = award_zip.namelist()[1:]
    median_list = []
    agency_list = []
    file_size_list = []
    row_count_list = []
    
    for i in range(len(file_names)):
        file_df = pd.read_csv(award_zip.open(file_names[i]), low_memory = False)
        file_df['period_of_performance_start_date'] = pd.to_datetime(file_df['period_of_performance_start_date'],
                                                                    errors = 'coerce')
        file_df['year'] = file_df['period_of_performance_start_date'].dt.year
        file_sum = file_df.groupby(['year'], as_index = False)['total_obligation'].sum()
        file_sum_median = file_sum['total_obligation'].median()
        
        median_list.append(file_sum_median)
        agency_list.append(file_df['funding_agency_id'][0])
        row_count_list.append(file_df.shape[0])
        
    for info in award_zip.infolist()[1:]:
        file_size_list.append(info.file_size)
        
    return median_list, agency_list, file_size_list, row_count_list

In [ ]:
zip_file = 'awards.zip'
median_list, agency_list, file_size_list, row_count_list = annual_median(zip_file)

In [ ]:
df = pd.DataFrame({'median': median_list, 'agency_id': agency_list, 'file_size': file_size_list,
                  'row_count': row_count_list})

df['median_log'] = np.log(df['median'])
df2 = df[df['median_log'] > 0]
df.sort_values(['median'], ascending = False).tail()

In [ ]:
df['median'].describe()

In [ ]:
plt.figure(figsize = (18, 7))

plt.subplot(1, 2, 1)
sns.distplot(df['median'], kde = False)
plt.title('Distribution Plot of Median Spending')
plt.xlabel('Spending')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
sns.boxplot(df['median'])
plt.title('Boxplot of Median Spending')
plt.xlabel('Spending')

plt.show()

In [ ]:
df2['median_log'].describe()

In [ ]:
plt.figure(figsize = (15, 7))

plt.subplot(1, 2, 1)
sns.distplot(df2['median_log'], kde = False)
plt.title('Distribution Plot of log(Median) Spending')
plt.xlabel('Spending')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
sns.boxplot(df2['median_log'])
plt.title('Boxplot of log(Median) Spending')
plt.xlabel('Spending')

plt.show()

In [ ]:
df['file_size'].describe()

In [ ]:
plt.figure(figsize = (15, 7))

plt.subplot(1, 2, 1)
sns.distplot(df['file_size'], kde = False)
plt.title('Distribution of File Size')
plt.xlabel('File Size')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
sns.boxplot(df['file_size'])
plt.title('Boxplot of File Size')
plt.xlabel('File Size')

plt.show()

In [ ]:
def head_tail(df, column, n_rows):
    """Sorts a dataframe by values according to the given column and returns both the head and tail of the dataframe
    
    Args:
        column (string): column name used to sort the dataframe
        n_rows (int)   : number of rows to return for each head and tail
        
    Returns:
        A dataframe of the head and tail rows of the initial dataframe
    """

    df = df.sort_values([column])
    df = df.head(n_rows).append(df.tail(n_rows))
    
    return df

head_tail(df, column = 'file_size', n_rows = 5)

In [ ]:
end_time = datetime.datetime.now()
run_time = end_time - start_time
time_difference_in_minutes = run_time / datetime.timedelta(minutes = 1)

print('time taken to process all the data in minutes (excluding 0.csv): ', time_difference_in_minutes)